In [ ]:
import boto3
import requests
from datetime import datetime

# === CONFIGURACIÓN ===
BUCKET = "headlinesjobs"  # Reemplaza con tu bucket
S3_PREFIX = "headlines/raw/"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# === SITIOS A DESCARGAR ===
sites = {
    "eltiempo": "https://www.eltiempo.com/",
}

# === FECHA ACTUAL ===
today = datetime.utcnow().strftime("%Y-%m-%d")

# === INICIALIZAR CLIENTE S3 ===
s3 = boto3.client("s3")

# === DESCARGAR Y GUARDAR HTML EN S3 ===
for site_name, url in sites.items():
    print(f"🔽 Descargando {url}")
    try:
        response = requests.get(url, headers=HEADERS, timeout=15)
        response.raise_for_status()
        html_content = response.text

        # Ruta de destino en S3
        filename = f"{S3_PREFIX}contenido-{site_name}-{today}.html"

        # Subir a S3
        s3.put_object(
            Bucket=BUCKET,
            Key=filename,
            Body=html_content.encode("utf-8"),
            ContentType="text/html"
        )
        print(f"✅ Guardado: s3://{BUCKET}/{filename}")

    except Exception as e:
        print(f"❌ Error al procesar {url}: {e}")
